In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

from setup_imports import *  # noqa: F401,F403


from src.story import get_story, get_all_stories
from src.challenges.models import ChallengeRecord
from langcodes import Language

COLLECTION = "LM2000"
DECK = "Pack03"

In [ ]:
target_lang = Language.get("sv-SE")
print(target_lang.display_name())

In [ ]:
stories = get_all_stories(collection=COLLECTION)

In [ ]:
story = get_story(story_title_hash="the_mystery_at_the_art_gallery_1df9ef")

In [ ]:
for s in stories:
    print(s.deck)

In [ ]:
challenge = ChallengeRecord.create(story)
challenge.upload()

In [ ]:
challenge.publish(target_language=target_lang)